In [1]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.

def get_crack_front_points(input_file):
    with open(input_file, 'r') as crack_points_file:
        crack_points_file.readline()
        crack_points = []
        for line in crack_points_file:
            tokens = line.split(',')
            crack_eyed_peas = tokens[0]
            theta = tokens[1]
            x = tokens[5]
            y = tokens[6]
            z = tokens[7].strip()
            dadN3Dline = tokens[3]
            # TODO: Get rid of this?
#             if int(crack_eyed_peas) > 0:
            crack_points.append((crack_eyed_peas, theta, x, y, z, dadN3Dline))
    return crack_points

In [2]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.

def get_crack_surface_points(input_file):
    with open(input_file, 'r') as crack_surface_points_file:
        crack_surface_points_file.readline()
        crack_surface_points = []
        for line in crack_surface_points_file:
            tokens = line.split(',')
            x = float(tokens[3])
            y = float(tokens[4])
            z = float(tokens[5])
            crack_surface_points.append((x, y, z))
    return crack_surface_points

In [3]:
# http://stackoverflow.com/questions/12141150/from-list-of-integers-get-number-closest-to-a-given-value

from bisect import bisect_left

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return after
    else:
        return before

In [5]:
from math import atan, pi, cos, sin
from scipy.spatial import cKDTree
import numpy as np

crack_front_points = get_crack_front_points('../../../data/0-crack_front_growth_rates_1500ppcf_transformed.csv')
    
thetas = []
crack_id_theta_to_xyz= []
for i in range(0,9):
    thetas.append([])
    crack_id_theta_to_xyz.append({})


for crack_front_point in crack_front_points:
    crack_id, theta, x, y, z, dadN3Dline = crack_front_point
    crack_id = int(crack_id)
    theta = float(theta)
    x = float(x)
    y = float(y)
    z = float(z)
#     if crack_id < 8:
    thetas[crack_id].append(theta)
    crack_id_theta_to_xyz[crack_id][theta] = (x,y,z)

# for i in range(0,8):
#     crack_id_theta_to_index = cKDTree(thetas[i])
    
crack_surface_points = get_crack_surface_points('../../../data/0-D1_grid_points_with_attributes_transformed.csv')
crack_surface_points = np.array(crack_surface_points)
surface_xy_pts = crack_surface_points[:,0:2]
kd_tree = cKDTree(surface_xy_pts)
print('done')

done


In [7]:
step_size = 2
pts_to_be_written = []
# first_crack_pt = True

# This reference point is from the 0-crack_front_growth_rates_1500ppcf_transformed.csv file for CF_id=0
reference_x = -1.9932
reference_y = 363.26

natee = 0
next_print = len(crack_front_points) / 10

for crack_front_point in crack_front_points:
    
    natee += 1
    if natee > next_print:
        print('Progress: ', (natee / len(crack_front_points)))
        next_print += len(crack_front_points) / 10
    
    # GET THE POINTS ON THE CRACK FRONT
    crack_id, theta, x, y, z, dadN3Dline = crack_front_point
    crack_id = int(crack_id)
    
    theta = float(theta)
    x = float(x)
    y = float(y)
    z = float(z)
    dadN3Dline = dadN3Dline.strip()
        
    if crack_id < 8:
        if crack_id != 0:
            pts_to_be_written.append((dadN3Dline, crack_id, theta, x, y, z, True))

            # CALCULATE THE REAL THETA FOR POINTS

            # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
            opposite_point = abs(x - reference_x)
            adjacent_point = abs(y - reference_y)

            # If the line isn't exactly vertical (i.e.: does not have undefined slope)
            if adjacent_point != 0:
                calculated_theta = atan(opposite_point / adjacent_point)

                # If the given point (x,y) is to the right of the reference point, then we got the wrong theta and we need to flip it
                if y > reference_y:
                    calculated_theta = pi - calculated_theta
            else:
                calculated_theta = pi / 2

            # CALCULATE THE DISTANCE FROM REFERENCE TO POINTS
            current_distance = ((x - reference_x)**2 + (y - reference_y)**2)**.5
            current_distance += step_size

        else: # crack_id == 0:

            # UPDATE X,Y
            theta_for_xy = takeClosest(thetas[crack_id+1], theta)
            stop_at_x, stop_at_y = crack_id_theta_to_xyz[crack_id+1][theta_for_xy][0:2]
            distance_to_next_crack_front = ((stop_at_x - reference_x)**2 + (stop_at_y - reference_y)**2)**.5

            # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
            opposite_point = abs(stop_at_x - reference_x)
            adjacent_point = abs(stop_at_y - reference_y)

            # If the line isn't exactly vertical (i.e.: does not have undefined slope)
            if adjacent_point != 0:
                calculated_theta = atan(opposite_point / adjacent_point)

                # If the given point (x,y) is to the right of the reference point, then we got the wrong theta and we need to flip it
                if stop_at_y > reference_y:
                    calculated_theta = pi - calculated_theta
                    
                if stop_at_x < reference_x:
                    calculated_theta = -calculated_theta
            else:
                calculated_theta = pi / 2

            if calculated_theta < pi / 2:
                next_y = reference_y - step_size * cos(calculated_theta)
                next_x = reference_x + step_size * sin(calculated_theta)
            else:
                temp_theta = pi - calculated_theta
                next_y = reference_y + step_size * cos(temp_theta)
                next_x = reference_x + step_size * sin(calculated_theta)

            # CALCULATE THE DISTANCE FROM REFERENCE TO POINTS
            current_distance = step_size

        theta_for_xy = takeClosest(thetas[crack_id+1], theta)
        stop_at_x, stop_at_y = crack_id_theta_to_xyz[crack_id+1][theta_for_xy][0:2]
        distance_to_next_crack_front = ((stop_at_x - reference_x)**2 + (stop_at_y - reference_y)**2)**.5

        # USE THIS TO GO FROM POINT ON CRACK FRONT OUT IN THE REAL THETA DIRECTION R DISTANCE

        while current_distance < distance_to_next_crack_front:
            if calculated_theta < pi / 2:
                next_y = reference_y - current_distance * cos(calculated_theta)
                if x >= reference_x:
                    next_x = reference_x + current_distance * sin(calculated_theta)
                else:
                    next_x = reference_x - current_distance * sin(calculated_theta)
            else:
                temp_theta = pi - calculated_theta
                next_y = reference_y + current_distance * cos(temp_theta)
                if x >= reference_x:
                    next_x = reference_x + current_distance * sin(calculated_theta)
                else:
                    next_x = reference_x - current_distance * sin(calculated_theta)

            # FIND THE Z FROM A KD TREE
            index = kd_tree.query((next_x, next_y))[1]
            next_z = crack_surface_points[index][2]

            # SAVE THE XYZ OF THESE POINTS
            pts_to_be_written.append((dadN3Dline, crack_id, theta, next_x, next_y, next_z, False))
            current_distance += step_size
    else: # crack_id < 8
        pts_to_be_written.append((dadN3Dline, crack_id, theta, x, y, z, True))

with open('../../../data/0-FINAL_TEST-points-surface.csv', 'w') as w:
    w.write('dadN,crack_id,theta,x,y,z,on_crack_front\n')
    for point in pts_to_be_written:
        dadN = point[0]
        crack_id = point[1]
        theta = point[2]
        x = point[3]
        y = point[4]
        z = point[5]
        on_crack_front = point[6]
        to_write = '{},{},{},{},{},{},{}\n'.format(dadN, crack_id, theta, x, y, z, on_crack_front)
        w.write(to_write)
print('done')

Progress:  0.10003087372645879
Progress:  0.20006174745291758
Progress:  0.3000154368632294
Progress:  0.40004631058968815
Progress:  0.500077184316147
Progress:  0.6000308737264588
Progress:  0.7000617474529176
Progress:  0.8000154368632294
Progress:  0.9000463105896882
done
